<a href="https://colab.research.google.com/github/Robertgaraban/Calculo-de-Estructuras-Navales-FNB/blob/main/Trabajo_final_EF_FNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ==============================================
# Ingeniería Estructural Naval FEM - Grúa de Buque
# Autor: Especialista en FEM Naval
# ==============================================

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

# ----------------------
# 1. Definición de Propiedades y Geometría
# ----------------------
E = 205e9  # Módulo de Young (Pa)
A_rect = (0.1 * 0.1) - ((0.1 - 0.002) * (0.1 - 0.002))  # m^2, sección rectangular 100x100x2 mm
A_circ = np.pi * ( (0.05)**2 - (0.048)**2 )  # m^2, sección circular 100x2 mm

# Definir nodos (en metros)
nodes = {
    1: np.array([0.0, 0.0]),
    2: np.array([2.0, 0.0]),
    3: np.array([2.0, 2.0]),
    4: np.array([0.0, 2.0])
}

# Definir elementos
# Formato: (Nodo inicio, Nodo fin, área, tipo)
elements = {
    'E1': (1, 2, A_rect, 'rectangular'),
    'E2': (2, 3, A_circ, 'circular'),
    'E3': (3, 4, A_rect, 'rectangular'),
    'E4': (4, 1, A_circ, 'circular')
}

# Carga en nodo 3 (vertical hacia abajo)
loads = np.zeros(2 * len(nodes))
loads[2 * (3-1) + 1] = -10000  # N

# Condiciones de borde: nodo 1 fijo, nodo 4 solo vertical libre
fixed_dofs = [0, 1, 2*3, 2*3+1]

# ----------------------
# 2. Ensamblaje Matriz de Rigidez Global
# ----------------------
ndof = 2 * len(nodes)
K_global = np.zeros((ndof, ndof))

for key, (n1, n2, A, tipo) in elements.items():
    x1, y1 = nodes[n1]
    x2, y2 = nodes[n2]
    L = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    c = (x2 - x1) / L
    s = (y2 - y1) / L
    k_local = (E * A / L) * np.array([
        [ c*c,  c*s, -c*c, -c*s],
        [ c*s,  s*s, -c*s, -s*s],
        [-c*c, -c*s,  c*c,  c*s],
        [-c*s, -s*s,  c*s,  s*s]
    ])
    idx = [2*(n1-1), 2*(n1-1)+1, 2*(n2-1), 2*(n2-1)+1]
    for i in range(4):
        for j in range(4):
            K_global[idx[i], idx[j]] += k_local[i, j]

# ----------------------
# 3. Aplicar Condiciones de Borde y Resolver
# ----------------------
# Eliminar DOFs fijos
free_dofs = list(set(range(ndof)) - set(fixed_dofs))

K_reduced = K_global[np.ix_(free_dofs, free_dofs)]
loads_reduced = loads[free_dofs]

# Verificación numérica avanzada
cond_num = np.linalg.cond(K_reduced)
if cond_num > 1e12:
    displacements = np.linalg.pinv(K_reduced) @ loads_reduced
else:
    displacements = np.linalg.solve(K_reduced, loads_reduced)

full_displacements = np.zeros(ndof)
full_displacements[free_dofs] = displacements

# Calcular Reacciones
reactions = K_global @ full_displacements - loads

# ----------------------
# 4. Cálculo de Fuerzas Axiales
# ----------------------
element_forces = {}
for key, (n1, n2, A, tipo) in elements.items():
    x1, y1 = nodes[n1]
    x2, y2 = nodes[n2]
    L = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    c = (x2 - x1) / L
    s = (y2 - y1) / L
    idx = [2*(n1-1), 2*(n1-1)+1, 2*(n2-1), 2*(n2-1)+1]
    u_elem = full_displacements[idx]
    axial_force = (E * A / L) * np.dot([-c, -s, c, s], u_elem)
    element_forces[key] = axial_force

# ----------------------
# 5. Visualización Avanzada
# ----------------------
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Configuración Original
for key, (n1, n2, _, tipo) in elements.items():
    x = [nodes[n1][0], nodes[n2][0]]
    y = [nodes[n1][1], nodes[n2][1]]
    color = '#1f77b4' if tipo == 'rectangular' else '#ff7f0e'
    ax1.plot(x, y, color=color, lw=2)

# Cargas
ax1.arrow(nodes[3][0], nodes[3][1], 0, -0.5, head_width=0.1, head_length=0.2, fc='k', ec='k')

# Apoyos
ax1.plot(nodes[1][0], nodes[1][1], 'ks', markersize=10)
ax1.plot(nodes[4][0], nodes[4][1], 'ko', markersize=10)

ax1.set_title('Configuración Original')
ax1.axis('equal')
ax1.set_xlabel('X [m]')
ax1.set_ylabel('Y [m]')

# Deformada
scale = 50
for key, (n1, n2, _, tipo) in elements.items():
    x = [nodes[n1][0] + scale * full_displacements[2*(n1-1)],
         nodes[n2][0] + scale * full_displacements[2*(n2-1)]]
    y = [nodes[n1][1] + scale * full_displacements[2*(n1-1)+1],
         nodes[n2][1] + scale * full_displacements[2*(n2-1)+1]]
    color = 'red' if element_forces[key] > 0 else 'blue'
    ax2.plot(x, y, color=color, lw=2)

ax2.set_title('Configuración Deformada x{}'.format(scale))
ax2.axis('equal')
ax2.set_xlabel('X [m]')
ax2.set_ylabel('Y [m]')

plt.suptitle('Análisis Estructural de Grúa Naval', fontsize=16)
plt.show()

# ----------------------
# 6. Resultados Finales
# ----------------------
print("\nDesplazamientos Nodos (mm):")
for i in range(len(nodes)):
    ux = full_displacements[2*i]*1e3
    uy = full_displacements[2*i+1]*1e3
    print(f"Nodo {i+1}: Ux = {ux:.3f} mm, Uy = {uy:.3f} mm")

print("\nReacciones en Apoyos (kN):")
for idx in fixed_dofs:
    print(f"GDL {idx+1}: {reactions[idx]/1e3:.2f} kN")

print("\nFuerzas Axiales Elementos (N):")
for key, force in element_forces.items():
    if abs(force) > 100:
        print(f"{key}: {force:.2f} N")


OSError: 'seaborn-darkgrid' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)